<a href="https://colab.research.google.com/github/felixsimard/comp551-p2/blob/main/P2_TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Part 2: Text Classification (20 points)**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import joblib
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Constants

contractions_dict = {"ain't": "are not","'s":" is","aren't": "are not"}
# Regular expression for finding contractions
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Define dataset paths
fake_new_train_dir = r'fake_news/fake_news_train.csv'
fake_news_val_dir = r'fake_news/fake_news_val.csv'
fake_news_test_dir = r'fake_news/fake_news_test.csv'

# Load datasets
fake_news_train = pd.read_csv(fake_new_train_dir, engine="python", error_bad_lines=False)
fake_news_val = pd.read_csv(fake_news_val_dir, engine="python", error_bad_lines=False)
fake_news_test = pd.read_csv(fake_news_test_dir, engine="python", error_bad_lines=False)

fake_news_train

,text,label
0,Indian fruit is so important to so many people...,0
1,"FORT WORTH, Texas — Urú Inc. will hold a confe...",0
2,"With three of the four new carriers, the Niger...",0
3,Let's start with the classic annual dividend r...,0
4,Following are some of the major events to have...,1
...,...,...
19995,"Warning: small, petty spoilers for the Game of...",1
19996,Shilpa Shetty will soon make her Bollywood deb...,0
19997,Add a digital black hole image to the Allstate...,0
19998,Share\nThe name W. L. Gore & Associates might ...,1


#### Preprocessing

In [3]:
# Stemmer function
def apply_stemmer(text):
    stemmer= PorterStemmer()
    return " ".join([stemmer.stem(word) for word in text.split()])

# Expanding contractions
# Reference: https://www.analyticsvidhya.com/blog/2021/06/must-known-techniques-for-text-preprocessing-in-nlp/
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

# Remove stop-words
# Reference: https://www.analyticsvidhya.com/blog/2021/06/must-known-techniques-for-text-preprocessing-in-nlp/
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

# Preprocessing function
def preprocess(df):

    # Lowercase everything
    df['text'] = df['text'].str.lower()

    # Expand contractions
    # Reference: https://www.analyticsvidhya.com/blog/2021/06/must-known-techniques-for-text-preprocessing-in-nlp/
    df['text'] = df['text'].apply(lambda x : expand_contractions(x))

    # Remove ponctuation
    df['text'] = df['text'].replace(r'[^\w\s]', r'', regex=True)

    # Remove numbers
    df['text'] = df['text'].replace(r'\d', r'', regex=True)

    # Remove special characters (eg: \n)
    df['text'] = df['text'].replace(r'\\[a-z]', r'', regex=True)

    # Remove stop words
    df['text'] = df['text'].apply(lambda x : remove_stopwords(x))

    # Stemming
    df["text"] = df["text"].apply(lambda x: apply_stemmer(x))

    # Trim whitespaces
    df['text'] = df['text'].str.strip()

    # Bag of words + Tokenization
    vect = CountVectorizer()
    X_train_counts = vect.fit_transform(df.text)
    joblib.dump(vect, "vectorizer.pkl")
    print("Vectorizer vocabulary:", vect.vocabulary_.get(u'algorithm'))
    print("Count Vectorizer shape:", X_train_counts.shape)
    

    # TF-IDF representation using bag-of-words matrix
    tfidf_transform = TfidfTransformer()
    X_train_tfidf = tfidf_transform.fit_transform(X_train_counts)
    joblib.dump(tfidf_transform, "tfidf_transform.pkl")
    print("TF-IDF shape:", X_train_tfidf.shape)
    print("\n")

    return X_train_tfidf, df

fake_news_train_preprocessed_tfidf, fake_news_train_preprocessed = preprocess(fake_news_train)
fake_news_train_preprocessed

Vectorizer vocabulary: 3155
Count Vectorizer shape: (20000, 138086)
TF-IDF shape: (20000, 138086)




,text,label
0,indian fruit import mani peopl season come end...,0
1,fort worth texa urú inc hold confer call today...,0
2,three four new carrier nigerian airspac manag ...,0
3,let start classic annual dividend reinvest pla...,0
4,follow major event occur apr reuter britain fr...,1
...,...,...
19995,warn small petti spoiler game throne season ei...,1
19996,shilpa shetti soon make bollywood debut opposi...,0
19997,add digit black hole imag allstat compani year...,0
19998,share name w l gore associ might immedi ring b...,1


In [4]:
fake_news_train_preprocessed.iloc[0]['text']

'indian fruit import mani peopl season come end first stop pick someth fresh tangi sugar chop papaya appl littl complic good fat come sugar well milk almond two great sourc fat well coupl fruit avocado okra result healthi delici appletoberri spread easi yummi delici oz appl appleliv bon bon tbsp sugar tsp almond extract freshli ground black pepper tbsp rice vinegar thick lightflavor bon bon season salt pepper tast vari thick finish add fruit bowl small mixer fit larg paddl attach line bowl nonstick cook spray use spatula meal add enough sugar almond extract shake excess mixtur thick solid remov bowl mixer slot spoon mix work spatula ballerina fold sugar need combin egg mixtur rice vinegar almond extract stir berri stirfri combin minut done serv nutrit inform per serv calori g fat g satur fat mg cholesterol g carbohydr g sugar g protein mg sodium g fiber view oil discov local salad dress instruct place shred bon bon food processor process smooth use sharp knife remov chunk place bowl fr

#### Model Training, Validation, Testing

In [5]:
def train_model(x_train, y_train):
    # preprocess x_train
    x_train_preprocessed_tfidf, x_train_preprocessed  = preprocess(x_train)

    # fit model
    model = LogisticRegression().fit(x_train_preprocessed_tfidf, y_train)
    training_score = model.score(x_train_preprocessed_tfidf, y_train)
    joblib.dump(train_model, "model.pkl")

    print("Training score:", training_score)

    return model

def transform_preprocess(x_test):
    # Load saved pickles
    loaded_vectorizer = joblib.load("vectorizer.pkl")
    loaded_tfidf_transform = joblib.load("tfidf_transform.pkl")
    loaded_model = joblib.load("model.pkl")

    # Transform
    x_val_vec = loaded_vectorizer.transform(x_test)
    x_val_tfidf = loaded_tfidf_transform.transform(x_val_vec)

    return x_val_tfidf


In [6]:
# Train model
trained_model = train_model(fake_news_train, fake_news_train['label'])

Vectorizer vocabulary: 3118
Count Vectorizer shape: (20000, 136944)
TF-IDF shape: (20000, 136944)


Training score: 0.84045


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [7]:
# Test set
x_test_tfidf = transform_preprocess(fake_news_test['text'])
y_predictions = trained_model.predict(x_test_tfidf)

acc_score = accuracy_score(fake_news_test['label'], y_predictions)
print("Accuracy on test set:", acc_score)

Accuracy on test set: 0.65
